In [1]:
import os
os.chdir("..")
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
import torch
torch.cuda.is_available()

In [2]:
from datasets import load_dataset_builder, load_dataset
dataset = load_dataset("truongpdd/vietnamese_poetry")

In [3]:
train_dataset = dataset['train']

# Mở file để ghi
with open('./train_data/vietnam_poetry.txt', 'w', encoding='utf-8') as f:
    for example in train_dataset:
        text = example['text']
        # Thêm token <s> vào đầu mỗi dòng
        f.write(f"<s>{text}\n")

print("Đã lưu tất cả text vào file text_output.txt")

Đã lưu tất cả text vào file text_output.txt


In [ ]:
with open('./data/text_output.txt', 'r', encoding='utf-8') as f:
    data = f.read()

In [ ]:
data[:1000]

In [ ]:
def create_conversation(example):
  # Xử lý để lấy prompt và completion từ dữ liệu của bạn
  # Ví dụ: lấy 50 từ đầu làm prompt
  words = example['text'].split()
  prompt = " ".join(words[:50])
  completion = " ".join(words[50:])

  # Tạo list of dicts cho cuộc hội thoại
  conversation = [
    {"role": "system", "content": f"Bạn là một chuyên gia về văn học Việt Nam. Hãy trả lời các câu hỏi về tác phẩm và tác giả một cách chính xác và sâu sắc."},
    {"role": "user", "content": f"Hãy viết tiếp đoạn văn sau:\n{prompt}"},
    {"role": "assistant", "content": completion}
  ]
  return conversation

In [ ]:
from transformers import AutoTokenizer

# Tải tokenizer của mô hình bạn chọn
model_name = "Qwen/Qwen2.5-0.5B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

def apply_template(example):
  # Tạo cuộc hội thoại
  conversation = create_conversation(example)
  # Áp dụng chat template để chuyển thành chuỗi văn bản
  example['formatted_text'] = tokenizer.apply_chat_template(conversation, tokenize=False)
  return example

# Áp dụng hàm trên cho toàn bộ dataset
formatted_dataset = dataset.map(apply_template)

In [ ]:
formatted_dataset['train'][1]

In [ ]:
tmp_dataset = formatted_dataset['train'].train_test_split(test_size=0.1)["test"]
tmp_dataset
# eval_dataset = formatted_dataset['test'].train_test_split(test_size=0.1)["test"]

In [ ]:
train_dataset = tmp_dataset.train_test_split(test_size=0.1)["train"]
eval_dataset = tmp_dataset.train_test_split(test_size=0.1)["test"]

In [ ]:
train_dataset

In [ ]:
eval_dataset

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, prepare_model_for_kbit_training
from trl import SFTTrainer
from datasets import DatasetDict
from peft import (
    LoraConfig,
    get_peft_model,
    TaskType,
    prepare_model_for_kbit_training
)

# Tải mô hình
# model_name = "Qwen/Qwen2.5-1.5B-Instruct"

# Load tokenizer


# Quantization configuration for memory efficiency
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# Load model with quantization
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    # device_map="cuda",
    trust_remote_code=True,
    torch_dtype=torch.bfloat16
).to('cuda' if torch.cuda.is_available() else 'cpu')

# Disable caching for training
model.config.use_cache = False

# Prepare model for k-bit training
model = prepare_model_for_kbit_training(model)


# LoRA configuration
lora_config = LoraConfig(
    r=4,                    # Rank
    lora_alpha=4,           # Alpha parameter for LoRA scaling
    target_modules=[         # Target modules for Qwen2.5
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj"
    ],
    lora_dropout=0.1,        # Dropout probability for LoRA layers
    bias="none",             # Bias type
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

In [ ]:
from transformers import TrainingArguments
output_dir = "./qwen-vietnamese-poetry-lora"
# Training arguments
training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    optim="adamw_bnb_8bit",
    num_train_epochs=10,
    eval_strategy="steps",
    eval_steps=0.2,
    logging_steps=10,
    warmup_steps=10,
    logging_dir="./logs",
    save_strategy="epoch",
    save_steps=0.2,
    max_steps=-1,
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
    report_to="none",
    run_name="qwen-vietnamese-poetry-finetune"
)

In [ ]:
# # Data collator for text generation
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
    pad_to_multiple_of=8
)

# Tokenization function
def tokenize_function(examples):
    # Tokenize the formatted text
    tokenized = tokenizer(
        examples["formatted_text"],
        truncation=True,
        padding=True,
        max_length=512,
        return_tensors=None
    )
    # Add labels for causal language modeling
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

# Apply tokenization to datasets
tokenized_train_dataset = train_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=train_dataset.column_names
)

tokenized_eval_dataset = eval_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=eval_dataset.column_names
)

# Initialize trainer with PEFT config
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    data_collator=data_collator,
    args=training_arguments,
    peft_config=lora_config,  # Add this back - it's still needed for SFTTrainer
)

In [ ]:
train_dataset

In [ ]:
def formatting_func(example):
    # Trả về một list chứa chuỗi văn bản đã được định dạng
    # Mỗi item trong list tương ứng với một mẫu huấn luyện
    return example['formatted_text']

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    args=training_arguments,
    peft_config=lora_config,
    formatting_func=formatting_func
)

In [ ]:
# Start training
print("Starting training...")
trainer.train()

# Save the final model
trainer.save_model()
tokenizer.save_pretrained(output_dir)

print(f"Training completed! Model saved to {output_dir}")

In [ ]:
# Function to test the fine-tuned model
def test_model(prompt, max_length=100):
    """Test the fine-tuned model with a Vietnamese poetry prompt"""
    conversation = [
      {"role": "system", "content": f"Bạn là một chuyên gia về văn học Việt Nam. Hãy trả lời các câu hỏi về tác phẩm và tác giả một cách chính xác và sâu sắc."},
      {"role": "user", "content": f"Hãy viết tiếp đoạn văn sau:\nthơ bảy chữ: \n{prompt}"}
    ]
    formatted_prompt = tokenizer.apply_chat_template(
        conversation,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **inputs,
        max_new_tokens=max_length
    )
    
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]


    return response

In [ ]:
import os
os.chdir("..")

In [ ]:
lora_path = "notebooks/qwen-vietnamese-poetry-lora"
os.listdir(lora_path)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig
import torch

# 1. Define paths for the original base model and the LoRA checkpoint
base_model_id = "Qwen/Qwen2.5-0.5B-Instruct" # Or any other base model ID
lora_path = "notebooks/qwen-vietnamese-poetry-lora"

# 2. Load the original base model
model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    torch_dtype=torch.bfloat16
).to('cuda' if torch.cuda.is_available() else 'cpu')

# 3. Load the LoRA adapter weights and attach them to the base model
model = PeftModel.from_pretrained(model, lora_path)

# 4. Load the tokenizer from the base model
tokenizer = AutoTokenizer.from_pretrained(base_model_id)

print("LoRA model loaded successfully!")

# You can now proceed to generate text as shown in the previous example
# The `generate` function will automatically use the combined weights.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Tải tokenizer của mô hình bạn chọn
model_name = "notebooks/qwen-vietnamese-poetry-lora/checkpoint-9630"
print(os.listdir(model_name))
base_model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# Function to test the fine-tuned model
def test_model(prompt, max_length=100):
    """Test the fine-tuned model with a Vietnamese poetry prompt"""
    conversation = [
      {"role": "system", "content": f"Hãy trả lời các câu hỏi một cách chính xác và sâu sắc."},
      {"role": "user", "content": f"{prompt}"}
    ]
    formatted_prompt = tokenizer.apply_chat_template(
        conversation,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **inputs,
        max_new_tokens=max_length
    )
    
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]


    return response

In [ ]:
res = test_model("Ai là tổng thống mỹ", max_length=256)
res

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
train_dataset[0]["text"]

In [ ]:
dataset

In [ ]:
output_file = 'output.txt'

# Open the file in write mode ('w') and specify the encoding
with open(output_file, 'w', encoding='utf-8') as f:
    # Iterate through the 'text' column of the 'train' split
    for text_entry in dataset['train']['text']:
        # Write the line to the file, prepending '<s>' and adding a newline character
        f.write(f'<s>{text_entry}\n\n')

print(f"Data successfully saved to {output_file}")